In [ ]:
!pip install pandas
!pip install numpy
!pip install tensorflow
!pip install sklearn

  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


In [1]:
train = r'/content/train_FD001.txt'

In [2]:
import pandas as pd

In [3]:
col_names = ['unit_number', 'time_in_cycles'] + [f'op_setting_{i}' for i in range(1,4)] + [f'sensor_{i}' for i in range(1,22)]
df_train = pd.read_csv('train_FD001.txt', sep='\s+', header=None, names=col_names)

In [21]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20631 entries, 0 to 20630
Data columns (total 27 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   unit_number     20631 non-null  int64  
 1   time_in_cycles  20631 non-null  int64  
 2   op_setting_1    20631 non-null  float64
 3   op_setting_2    20631 non-null  float64
 4   op_setting_3    20631 non-null  float64
 5   sensor_1        20631 non-null  float64
 6   sensor_2        20631 non-null  float64
 7   sensor_3        20631 non-null  float64
 8   sensor_4        20631 non-null  float64
 9   sensor_5        20631 non-null  float64
 10  sensor_6        20631 non-null  float64
 11  sensor_7        20631 non-null  float64
 12  sensor_8        20631 non-null  float64
 13  sensor_9        20631 non-null  float64
 14  sensor_10       20631 non-null  float64
 15  sensor_11       20631 non-null  float64
 16  sensor_12       20631 non-null  float64
 17  sensor_13       20631 non-null 

In [7]:
max_cycle = df_train.groupby('unit_number')['time_in_cycles'].max()
df_train['RUL'] = df_train.apply(lambda row: max_cycle[row.unit_number] - row.time_in_cycles, axis=1)
df_train = df_train.merge(max_cycle, on='unit_number', suffixes=('', '_max'))
df_train.drop('time_in_cycles_max', axis=1, inplace=True)

In [19]:
df_train

,unit_number,time_in_cycles,op_setting_1,op_setting_2,op_setting_3,sensor_1,sensor_2,sensor_3,sensor_4,sensor_5,...,sensor_13,sensor_14,sensor_15,sensor_16,sensor_17,sensor_18,sensor_19,sensor_20,sensor_21,RUL
0,1,1,-0.0007,-0.0004,100.0,518.67,641.82,1589.70,1400.60,14.62,...,2388.02,8138.62,8.4195,0.03,392,2388,100.0,39.06,23.4190,191.0
1,1,2,0.0019,-0.0003,100.0,518.67,642.15,1591.82,1403.14,14.62,...,2388.07,8131.49,8.4318,0.03,392,2388,100.0,39.00,23.4236,190.0
2,1,3,-0.0043,0.0003,100.0,518.67,642.35,1587.99,1404.20,14.62,...,2388.03,8133.23,8.4178,0.03,390,2388,100.0,38.95,23.3442,189.0
3,1,4,0.0007,0.0000,100.0,518.67,642.35,1582.79,1401.87,14.62,...,2388.08,8133.83,8.3682,0.03,392,2388,100.0,38.88,23.3739,188.0
4,1,5,-0.0019,-0.0002,100.0,518.67,642.37,1582.85,1406.22,14.62,...,2388.04,8133.80,8.4294,0.03,393,2388,100.0,38.90,23.4044,187.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20626,100,196,-0.0004,-0.0003,100.0,518.67,643.49,1597.98,1428.63,14.62,...,2388.26,8137.60,8.4956,0.03,397,2388,100.0,38.49,22.9735,4.0
20627,100,197,-0.0016,-0.0005,100.0,518.67,643.54,1604.50,1433.58,14.62,...,2388.22,8136.50,8.5139,0.03,395,2388,100.0,38.30,23.1594,3.0
20628,100,198,0.0004,0.0000,100.0,518.67,643.42,1602.46,1428.18,14.62,...,2388.24,8141.05,8.5646,0.03,398,2388,100.0,38.44,22.9333,2.0
20629,100,199,-0.0011,0.0003,100.0,518.67,643.23,1605.26,1426.53,14.62,...,2388.23,8139.29,8.5389,0.03,395,2388,100.0,38.29,23.0640,1.0


In [53]:
df_train_y = df_train['RUL']
df_train_X = df_train.drop(['unit_number', 'RUL', 'time_in_cycles'], axis=1)

In [54]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

In [55]:
scaled_X = scaler.fit_transform(df_train_X)

In [56]:
pd.DataFrame(scaled_X).info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20631 entries, 0 to 20630
Data columns (total 24 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   0       20631 non-null  float64
 1   1       20631 non-null  float64
 2   2       20631 non-null  float64
 3   3       20631 non-null  float64
 4   4       20631 non-null  float64
 5   5       20631 non-null  float64
 6   6       20631 non-null  float64
 7   7       20631 non-null  float64
 8   8       20631 non-null  float64
 9   9       20631 non-null  float64
 10  10      20631 non-null  float64
 11  11      20631 non-null  float64
 12  12      20631 non-null  float64
 13  13      20631 non-null  float64
 14  14      20631 non-null  float64
 15  15      20631 non-null  float64
 16  16      20631 non-null  float64
 17  17      20631 non-null  float64
 18  18      20631 non-null  float64
 19  19      20631 non-null  float64
 20  20      20631 non-null  float64
 21  21      20631 non-null  float64
 22

In [70]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dropout
from tensorflow.keras.regularizers import l2

In [113]:
model = Sequential(
    [
    LSTM(64, activation='relu', return_sequences=True, input_shape=(30,24)),
    Dropout(0.3),
    LSTM(32, activation='relu', return_sequences=False, kernel_regularizer=l2(1e-4)),
    Dense(1, activation='linear')
    ]
)

In [114]:
model.summary()

Model: "sequential_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_12 (LSTM)                  │ (None, 30, 64)         │        22,784 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_31 (Dropout)            │ (None, 30, 64)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_13 (LSTM)                  │ (None, 32)             │        12,416 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_26 (Dense)                │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 35,233 (137.63 KB)

 Trainable params: 35,233 (137.63 KB)

 Non-trainable params: 0 (0.00 B)

In [115]:
X = np.array(scaled_X)
y = np.array(df_train_y)

In [116]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.00001), loss=tf.keras.losses.MeanSquaredError(), metrics=['mse'])

In [117]:
def create_sequences_per_unit(df, time_steps=30):
    Xs, ys = [], []
    for unit in df['unit_number'].unique():
        unit_df = df[df['unit_number'] == unit]
        unit_X = scaler.fit_transform(unit_df.drop(['unit_number', 'RUL', 'time_in_cycles'], axis=1))
        unit_y = unit_df['RUL'].values
        for i in range(len(unit_X) - time_steps):
            Xs.append(unit_X[i:i+time_steps])
            ys.append(unit_y[i+time_steps])
    return np.array(Xs), np.array(ys)

X_seq, y_seq = create_sequences_per_unit(df_train, time_steps=30)

In [118]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(
    X_seq, y_seq, test_size=0.2, shuffle=False
)

In [119]:
pd.DataFrame(scaled_X).info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20631 entries, 0 to 20630
Data columns (total 24 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   0       20631 non-null  float64
 1   1       20631 non-null  float64
 2   2       20631 non-null  float64
 3   3       20631 non-null  float64
 4   4       20631 non-null  float64
 5   5       20631 non-null  float64
 6   6       20631 non-null  float64
 7   7       20631 non-null  float64
 8   8       20631 non-null  float64
 9   9       20631 non-null  float64
 10  10      20631 non-null  float64
 11  11      20631 non-null  float64
 12  12      20631 non-null  float64
 13  13      20631 non-null  float64
 14  14      20631 non-null  float64
 15  15      20631 non-null  float64
 16  16      20631 non-null  float64
 17  17      20631 non-null  float64
 18  18      20631 non-null  float64
 19  19      20631 non-null  float64
 20  20      20631 non-null  float64
 21  21      20631 non-null  float64
 22

In [120]:
model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=30,
    batch_size=64,
    callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=2, restore_best_weights=True)]
    )

Epoch 1/30
221/221 ━━━━━━━━━━━━━━━━━━━━ 12s 35ms/step - loss: 11602.5830 - mse: 11602.5742 - val_loss: 16312.3857 - val_mse: 16312.3770
Epoch 2/30
221/221 ━━━━━━━━━━━━━━━━━━━━ 10s 35ms/step - loss: 11535.3154 - mse: 11535.3076 - val_loss: 16200.0576 - val_mse: 16200.0508
Epoch 3/30
221/221 ━━━━━━━━━━━━━━━━━━━━ 8s 37ms/step - loss: 10653.4873 - mse: 10653.4785 - val_loss: 5402.1646 - val_mse: 5402.1567
Epoch 4/30
221/221 ━━━━━━━━━━━━━━━━━━━━ 7s 33ms/step - loss: 3455.4695 - mse: 3455.4612 - val_loss: 4342.1729 - val_mse: 4342.1646
Epoch 5/30
221/221 ━━━━━━━━━━━━━━━━━━━━ 10s 33ms/step - loss: 2781.5835 - mse: 2781.5750 - val_loss: 5981.0620 - val_mse: 5981.0522
Epoch 6/30
221/221 ━━━━━━━━━━━━━━━━━━━━ 8s 38ms/step - loss: 2351.1333 - mse: 2351.1245 - val_loss: 2791.8115 - val_mse: 2791.8032
Epoch 7/30
221/221 ━━━━━━━━━━━━━━━━━━━━ 8s 37ms/step - loss: 2038.9723 - mse: 2038.9640 - val_loss: 3070.7141 - val_mse: 3070.7063
Epoch 8/30
221/221 ━━━━━━━━━━━━━━━━━━━━ 7s 32ms/step - loss: 1971.1626

In [121]:
model.save('turbofan_engine_model_updated.h5')